In [1]:
from google.colab import files

uploaded = files.upload() 

Saving alice.txt to alice.txt


In [2]:
!ls

alice.txt  datalab


In [3]:
!pip install keras

In [4]:
from keras.models import Sequential
from keras.layers import Dense,Activation
from keras.layers.recurrent import SimpleRNN
import numpy as np

Using TensorFlow backend.


In [0]:
fin=open('alice.txt',encoding='utf-8-sig')
lines=[]
for line in fin:
  line = line.strip().lower()
  #line = line.decode("ascii","ignore")
  if(len(line)==0):
    continue
  lines.append(line)
fin.close()
text = " ".join(lines)

In [6]:
text[:100]

'alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to'

In [0]:
import re
text = text.lower()
text = re.sub('[^0-9a-zA-Z]+',' ',text)

In [0]:
from collections import Counter
counts = Counter()
counts.update(text.split())
words = sorted(counts, key=counts.get, reverse=True)
chars = words
nb_chars = len(text.split())
char2index = {word: i for i, word in enumerate(chars)}
index2char = {i: word for i, word in enumerate(chars)}

In [9]:
SEQLEN = 10
STEP = 1
input_chars = []
label_chars = []
text2=text.split()
for i in range(0,nb_chars-SEQLEN,STEP):
    x=text2[i:(i+SEQLEN)]
    y=text2[i+SEQLEN]
    input_chars.append(x)
    label_chars.append(y)
print(input_chars[0])
print(label_chars[0])

['alice', 'was', 'beginning', 'to', 'get', 'very', 'tired', 'of', 'sitting', 'by']
her


In [10]:
total_chars = len(set(chars))
total_chars

3028

In [11]:
X = np.zeros((len(input_chars), SEQLEN, total_chars), dtype=np.bool)
y = np.zeros((len(input_chars), total_chars), dtype=np.bool)
# Create encoded vectors for the input and output values
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i,char2index[label_chars[i]]]=1
print(X.shape)
print(y.shape)

(30407, 10, 3028)
(30407, 3028)


In [0]:
HIDDEN_SIZE = 128
BATCH_SIZE = 1024
NUM_ITERATIONS = 100
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

In [13]:
model = Sequential()
model.add(SimpleRNN(HIDDEN_SIZE,return_sequences=False,input_shape=(SEQLEN,total_chars),unroll=True))
model.add(Dense(total_chars, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 128)               404096    
_________________________________________________________________
dense_1 (Dense)              (None, 3028)              390612    
Total params: 794,708
Trainable params: 794,708
Non-trainable params: 0
_________________________________________________________________


In [17]:
for iteration in range(50):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    # Fitting the values
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Time to see how our predictions fare
    # We are creating a test set from a random location in our dataset
    # In the code below, we are selecting a random input as our seed value of words
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    print("Generating from seed: %s" % (test_chars))
    print(test_chars)
    # From the seed words, we are tasked to predict the next words
    # In the code below, we are predicting the next 100 words (NUM_PREDS_PER_EPOCH) after the seed words
    for i in range(NUM_PREDS_PER_EPOCH):
        # Pre processing the input data, just like the way we did before training the model
        Xtest = np.zeros((1, SEQLEN, total_chars))
        for i, ch in enumerate(test_chars):
            Xtest[0, i, char2index[ch]] = 1
        # Predict the next word
        pred = model.predict(Xtest, verbose=0)[0]
        # Given that, the predictions are probability values, we take the argmax to fetch the location of highest probability
        # Extract the word belonging to argmax
        ypred = index2char[np.argmax(pred)]
        print(ypred,end=' ')
        # move forward with test_chars + ypred so that we use the original 9 words + prediction for the next prediction
        test_chars = test_chars[1:] + [ypred]

Iteration #: 0
Epoch 1/1
30407/30407 [==============================] - 3s 98us/step - loss: 2.4069
Generating from seed: ['what', 'to', 'say', 'to', 'this', 'so', 'she', 'helped', 'herself', 'to']
['what', 'to', 'say', 'to', 'this', 'so', 'she', 'helped', 'herself', 'to']
the house and yet and butter and then out at the queen and the mock turtle thought alice thought alice she herself to herself as well as she was quite at all but the hatter i ll have to go to see it she was ready to be to you like you i m sure you know you mean like you i don t you know you said the king but said to be like at the cat the little said alice king and alice for quite for it very all oh down and i can the thing of you ==================================================
Iteration #: 1
Epoch 1/1
30407/30407 [==============================] - 3s 97us/step - loss: 2.3729
Generating from seed: ['no', 'longer', 'to', 'be', 'seen', 'she', 'found', 'herself', 'in', 'a']
['no', 'longer', 'to', 'be', 'seen', 'she',

30407/30407 [==============================] - 3s 97us/step - loss: 2.3245
Generating from seed: ['m', 'growing', 'you', 've', 'no', 'right', 'to', 'grow', 'here', 'said']
['m', 'growing', 'you', 've', 'no', 'right', 'to', 'grow', 'here', 'said']
the cook she said to alice said alice alice as she could it at last she she as to herself out i m not know but i know what i ll have you said alice and she added to say it s not a little girl it i don t like it said alice nothing to herself it said the hatter i can t be join the dance i can be no for the dance will you do not agree will you d to the other you received 1 e of do not cried you or any the project gutenberg ==================================================
Iteration #: 3
Epoch 1/1
30407/30407 [==============================] - 3s 97us/step - loss: 2.3037
Generating from seed: ['hastily', 'replied', 'at', 'least', 'at', 'least', 'i', 'mean', 'what', 'i']
['hastily', 'replied', 'at', 'least', 'at', 'least', 'i', 'mean', 'what', 'i'

was she had been about round her head out and that there was very long again now it as ==================================================
Iteration #: 5
Epoch 1/1
30407/30407 [==============================] - 3s 98us/step - loss: 2.2295
Generating from seed: ['that', 'only', 'the', 'march', 'hare', 'said', 'i', 'didn', 't', 'the']
['that', 'only', 'the', 'march', 'hare', 'said', 'i', 'didn', 't', 'the']
mouse s the way alice said alice said the king was a a here of the queen and then would be like the duchess she felt to get her way after the she was going on the rabbit came began in the distance and seemed to herself what for meaning to the hatter went on in a little little went on in a great hurry who thought it all as well as that she had quite to her again and she soon was back on again before as she was very over to herself it i don t know what ==================================================
Iteration #: 6
Epoch 1/1
30407/30407 [==============================] - 3s 98us/step -

30407/30407 [==============================] - 3s 96us/step - loss: 2.1563
Generating from seed: ['do', 'such', 'a', 'thing', 'before', 'and', 'he', 'wasn', 't', 'going']
['do', 'such', 'a', 'thing', 'before', 'and', 'he', 'wasn', 't', 'going']
to look at the words and tried she had looking at into and in a little of of the duchess and this back again that she came upon out to the game said the march hare he thought alice in a little thing the queen was lying at all about it she looked at the or went on but she had on her time was a curious as she was up to see and she felt that to get to do it i m do you re all them of the way said the gryphon s made to look again ==================================================
Iteration #: 8
Epoch 1/1
30407/30407 [==============================] - 3s 96us/step - loss: 2.1140
Generating from seed: ['up', 'said', 'the', 'queen', 'in', 'a', 'shrill', 'loud', 'voice', 'and']
['up', 'said', 'the', 'queen', 'in', 'a', 'shrill', 'loud', 'voice', 'and']
t

30407/30407 [==============================] - 3s 97us/step - loss: 2.0584
Generating from seed: ['see', 'anything', 'that', 'looked', 'like', 'the', 'right', 'thing', 'to', 'eat']
['see', 'anything', 'that', 'looked', 'like', 'the', 'right', 'thing', 'to', 'eat']
and when she was in her hands but alice thought alice could herself to herself she was this for i wonder you know not about the right of your majesty she was not to herself and should herself said said the mock turtle who made what you alice said the gryphon i ve got to me at alice said alice as she said the caterpillar all in her hands all at last she was no to get down it ll be thought alice as she s sure about all it for be as well as it s no like all ==================================================
Iteration #: 11
Epoch 1/1
30407/30407 [==============================] - 3s 97us/step - loss: 2.0082
Generating from seed: ['the', 'time', 'she', 'went', 'on', 'talking', 'dear', 'dear', 'how', 'queer']
['the', 'time', 'she',

...

the next witness was a three all however she was his very anxiously of her head by little shriek of the court of no one and the gryphon and in her on close being her yes a the glass who and was getting would and this the other was head and found a very little voice over with and there gave that said the hatter thing the hatter then not you tell about and it would be like to they your like you don t think said the hatter remarked talk said the ==================================================
Iteration #: 38
Epoch 1/1
30407/30407 [==============================] - 3s 97us/step - loss: 1.1997
Generating from seed: ['the', 'stick', 'running', 'a', 'very', 'little', 'way', 'forwards', 'each', 'time']
['the', 'stick', 'running', 'a', 'very', 'little', 'way', 'forwards', 'each', 'time']
and a long eyes back and barking hoarsely all the whole thing of not like of do a work or with anything or tell it is this moment to know if you am but she was no after time to find and then they were made o

30407/30407 [==============================] - 3s 97us/step - loss: 1.1547
Generating from seed: ['anxiously', 'round', 'to', 'make', 'out', 'who', 'was', 'talking', 'alice', 'could']
['anxiously', 'round', 'to', 'make', 'out', 'who', 'was', 'talking', 'alice', 'could']
see out anything that s a little tea of instantly put it and looking then talking to be of for this for she was got alice to the footman and side so first as she got in the same words not just yes you can the the door i can t take his more know the duchess i don t know it what i don t said alice said alice she added to have but i a they s said the hatter said the hatter there would the whole well and alice added in all a pack is what you ==================================================
Iteration #: 41
Epoch 1/1
30407/30407 [==============================] - 3s 97us/step - loss: 1.1183
Generating from seed: ['you', 'did', 'said', 'the', 'mock', 'turtle', 'hold', 'your', 'tongue', 'added']
['you', 'did', 'said', 'the', 

the house and seemed anything the march hare you had to well my when it was more over ==================================================
Iteration #: 43
Epoch 1/1
30407/30407 [==============================] - 3s 97us/step - loss: 1.0797
Generating from seed: ['beautiful', 'beautiful', 'soup', 'chapter', 'xi', 'who', 'stole', 'the', 'tarts', 'the']
['beautiful', 'beautiful', 'soup', 'chapter', 'xi', 'who', 'stole', 'the', 'tarts', 'the']
king and queen of hearts were seated on their throne when they arrived with a great crowd assembled about them all sorts of little children and beasts as well as the whole thing said to the end of in a from but the dodo all first fees that is as the dormouse said nothing as it oh no me enough for it s what such the cat little as the queen of another watch while and in such a tone of of of and said alice and went on in long words the cat on no before at everything about ==================================================
Iteration #: 44
Epoch 1/1
30407/3

30407/30407 [==============================] - 3s 98us/step - loss: 1.0348
Generating from seed: ['of', 'living', 'would', 'be', 'like', 'but', 'it', 'puzzled', 'her', 'too']
['of', 'living', 'would', 'be', 'like', 'but', 'it', 'puzzled', 'her', 'too']
much so she went on but as do as she she just it eagerly to find the queen but she had to herself down down again and she did it the for her way down she found on her of his voice of now like the she went on you were not than here now is a she said to herself she felt me how to be me for at this time she s a long tone and then by was could round it again i i ve i think you do better said alice i don t talk that ==================================================
Iteration #: 46
Epoch 1/1
30407/30407 [==============================] - 3s 97us/step - loss: 1.0095
Generating from seed: ['a', 'little', 'timidly', 'why', 'you', 'are', 'painting', 'those', 'roses', 'five']
['a', 'little', 'timidly', 'why', 'you', 'are', 'painting', 'those', 'ro

and he went on in head out sure she had to go for the sea and looked another alice in its more under that dear did the ==================================================
Iteration #: 48
Epoch 1/1
30407/30407 [==============================] - 3s 96us/step - loss: 0.9646
Generating from seed: ['beast', 'screamed', 'off', 'with', 'her', 'head', 'off', 'nonsense', 'said', 'alice']
['beast', 'screamed', 'off', 'with', 'her', 'head', 'off', 'nonsense', 'said', 'alice']
very loudly and decidedly and the king was silent the dormouse laid his hand and alice as she spoke out foot what was were to talk and i think what did not like the cat that as she said to herself she said the cat one had made a moment and went on her there was a glass which in out what or they were it little dream and dormouse a the queen of of the mock turtle which found the same or birds it while it was the hatter alice i never think very curious all the place it ==================================================
Iteration

In [16]:
for i in range(10):
  print(i,end=' ')

0 1 2 3 4 5 6 7 8 9 